# Sequential Monte Carlo (SMC) の全体像

このページの目的は、コードを読む前に  
**「SMCが頭の中でどう動いているのか」**というモデルを作ることです。

数式の厳密な導出や理論的証明は扱わず、  
粒子がどのように動き、なぜその処理が必要なのかを  
**直感的に理解できること**を目標とします。

---

## このページで伝えたいこと

Sequential Monte Carlo（SMC）は、
<b>粒子集団でパラメータを追跡することで、近似的な分布を得る手法</b>
です。

MCMCと比較したときの最大の違いは、以下の2点にあります。

- 単一のサンプルではなく、**粒子集団**を同時に扱う
- 分布が「時間（ステップ）」とともに**徐々に変化**していく

この構造により、SMCでは  
1回の実行で **分布全体とその不確実性** を同時に得ることができます。

また、SMC法には大別して2つの手法があります。それぞれ
- data tempering
- likelihood tempering (bridge approach)
と呼ばれています。

data tempering は徐々にデータを追加し，分布を遷移させていく手法です．時系列データに適しており，データを追加する順序によって推定精度に影響が生じます．
likelihood tempering は data tempering よりも利用例が少ない手法で、尤度の影響を徐々に強めていくことで，滑らかに事前分布から事後分布へと遷移させていく手法です。
---

## SMCを構成する基本要素

### Particle
粒子とは、パラメータ空間上のサンプルです。  
SMCでは、1つの点ではなく多数の粒子の集合を使って分布を表現します。

### 重み

各粒子には、その粒子が  
「現在の目標分布にどれだけ適合しているか」を表す重みが割り当てられます。
具体的には尤度を用いて、それを行います。

時間が進むにつれて、この重みは偏っていく傾向があるため、工夫が必要になります。


### Resampling

重みが極端に偏ると、多くの粒子が実質的に意味を持たなくなります。  
これを防ぐために、有効な粒子を複製し、  
無効な粒子を捨てる操作がResamplingです。
このとき、極端に重みの大きな粒子が存在すると、粒子集団の多様性が失われ<b>縮退</b>と呼ばれる現象が起こります。
これを防ぐために、ESSという指標が重要になっており、私たちのアルゴリズムではこのESSに基づいて尤度の影響を変化させていきます。

### 事前分布・中間分布・事後分布

SMCでは、最初から難しい分布を直接扱いません。

- 最初は「簡単な分布」
- 徐々に「目的とする難しい分布」

へと移動していきます。

この変化は、温度（tempering）やデータ数の増加によって実現されます。

---

## SMCのフロー

SMCは、大きく4つのステップで構成されています。
1. Initialization
2. Likelihood calculation
3. Resampling
4. Mutation

始めにInitializationを行った後、2~4を繰り返すことで事後分布を得ます。

### 1. Initialization

Initialization では初めに事前分布𝑝(𝜽) を定義し，𝑝(𝜽) から粒子をサンプリングします。

### 2. Likelihood calculation

粒子の持つパラメータを用いて、尤度計算を行います。そして、各粒子の尤度を用いてそれぞれの重みを計算します。

### 3. Resampling

粒子の持つ重みに応じて、確率的に粒子を複製・消失させます。
例えば、全体の粒子数がNpであり、粒子全体で重みの正規化を行っているという条件を考えてみます。
このとき、重みが1/2Npの粒子は50%の確率で消失し、50%の確率で存続します。
重みが1.5/Npの粒子の場合は50%の確率で2つに複製され、50%の確率で1つのままになります。

### 4. Mutation

Resampling後の粒子は、同一パラメータを持つ集団が複数存在します。
そこで、各粒子に対してMetropolis Hasting法を行い、確率的に粒子の遷移を行います。

---



### 重みはなぜ偏るのか

分布が変化すると、  
一部の粒子だけが新しい分布に適合し、  
それ以外の粒子の重みは急激に小さくなります。

この現象を「粒子退化」と呼びます。

### なぜ再サンプリングが必要なのか

粒子退化が進むと、  
「粒子数は多いが、実質的には数個しか意味を持たない」  
状態になります。

再サンプリングは、この問題を回避するために不可欠な操作です。

---

## Likelihood tempering と Data tempering

SMCでは、「分布を徐々に難しくする」方法として  
主に以下の2つが使われます。

### Data tempering

データを少数から徐々に追加していき、  
観測情報を段階的に強くしていく方法です。

どちらも、

**「簡単な分布から難しい分布へ移動する」**

という点で本質的には同じ考え方に基づいています。

具体的な制御方法については、次のページで詳しく説明します。

### Likelihood tempering

尤度に指数（温度）をかけ、  
その指数を 0 から 1 に徐々に増やしていく方法です。

---

## SMCの出力

SMCを実行することで、以下の情報が得られます。

- パラメータの事後分布
- 推定値の不確実性（分散・信用区間など）
- 正規化定数（Evidence）

これは、点推定だけを行う手法とは大きく異なる利点です。

---

## 本コードで提供するもの

本サイトで扱うSMCコードは、以下を目的として設計されています。

- 汎用的に使えるSMCフレームワーク
- Likelihood tempering / Data tempering の切り替え
- 並列計算を前提とした設計
- 研究用途を意識した再現性の確保

次のページでは、  
これらの挙動を制御する **ハイパーパラメータ** について説明します。